In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.optim as optim
from glob import glob
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from nnspike.models import NvidiaModel
from nnspike.data import NvidiaDataset
from nnspike.constant import ROI_CNN
from scripts.tools import view_data_distribution

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load Training Data

In [ ]:
label_paths = glob("../storage/frames/*.csv")

df = pd.DataFrame()
for label_path in label_paths:
    label_df = pd.read_csv(label_path)
    df = pd.concat([df, label_df])

print(f"Total number of training records: {len(df)}")

In [ ]:
intervals = [2.3, 3]

df = df[df['interval'].isin(intervals) & df['use']==True]
df = df.reset_index(drop=True)  # Reset index for future data balancing

model_or_opencv = np.where(df['predicted_x'].isna(), df['mx'], df['predicted_x'])
df['offset_x'] = np.where(df['adjusted_x'].isna(), model_or_opencv, df['adjusted_x'])

print(f"The total number of samples is {len(df)}")
view_data_distribution(df)

### Load to pytorch Dataset

In [ ]:
image_paths = df['image_path'].to_list()
intervals = df['interval'].to_list()
courses = df['course'].to_list()
offset_xs = df['offset_x'].to_list()

X_all = [[x, y, z] for x, y, z in zip(image_paths, intervals, courses)]
y_all = offset_xs

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=0.2, random_state=6)

train_set = NvidiaDataset(inputs=X_train, offset_xs=y_train, roi=ROI_CNN, train_course="right")
val_set = NvidiaDataset(inputs=X_val, offset_xs=y_val, roi=ROI_CNN, train_course="right")

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=True)

### Model, Loss Function, Optimizer Definiation

In [ ]:
criterion = nn.MSELoss()
model = NvidiaModel()  # or NvidiaModelV2()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

### Training Loop

In [ ]:
# Initialize TensorBoard writer
writer = SummaryWriter()

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        optimizer.zero_grad()
        inputs = [input_tensor.to(device) for input_tensor in inputs]
        labels = labels.to(device)
        outputs = model(inputs[0])  # NvidiaModelV2(): outputs = model(inputs[0], inputs[1])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    # Evaluation
    for inputs, labels in val_loader:
        inputs = [input_tensor.to(device) for input_tensor in inputs]
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(inputs[0])  # NvidiaModelV2() : outputs = model(inputs[0], inputs[1])
        loss = criterion(outputs, labels)
        val_loss += loss.item()

    # # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)

    # Write to TensorBoard
    writer.add_scalar('Loss/train', avg_train_loss, epoch)
    writer.add_scalar('Loss/val', avg_val_loss, epoch)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.5f}, Val Loss: {avg_val_loss:.5f}')

# Close TensorBoard writer
writer.close()

### Save Model

In [ ]:
torch.save(model.state_dict(), "../storage/models/model_xx.pth")